In [1]:
# !wget https://huseinhouse-data.s3-ap-southeast-1.amazonaws.com/news.zip

In [2]:
# !unzip news.zip

In [3]:
from glob import glob

news = glob('news/*.json')
len(news)

461

In [4]:
import re
from unidecode import unidecode

alphabets = '([A-Za-z])'
prefixes = (
    '(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt|Puan|puan|Tuan|tuan|sir|Sir)[.]'
)
suffixes = '(Inc|Ltd|Jr|Sr|Co)'
starters = '(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever|Dia|Mereka|Tetapi|Kita|Itu|Ini|Dan|Kami)'
acronyms = '([A-Z][.][A-Z][.](?:[A-Z][.])?)'
websites = '[.](com|net|org|io|gov|me|edu|my)'
another_websites = '(www|http|https)[.]'
digits = '([0-9])'

def cleaning(string):
    string = unidecode(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

def split_into_sentences(text):
    text = unidecode(text)
    text = ' ' + text + '  '
    text = text.replace('\n', ' ')
    text = re.sub(prefixes, '\\1<prd>', text)
    text = re.sub(websites, '<prd>\\1', text)
    text = re.sub(another_websites, '\\1<prd>', text)
    if '...' in text:
        text = text.replace('...', '<prd><prd><prd>')
    if 'Ph.D' in text:
        text = text.replace('Ph.D.', 'Ph<prd>D<prd>')
    text = re.sub('\s' + alphabets + '[.] ', ' \\1<prd> ', text)
    text = re.sub(acronyms + ' ' + starters, '\\1<stop> \\2', text)
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]' + alphabets + '[.]',
        '\\1<prd>\\2<prd>\\3<prd>',
        text,
    )
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]', '\\1<prd>\\2<prd>', text
    )
    text = re.sub(' ' + suffixes + '[.] ' + starters, ' \\1<stop> \\2', text)
    text = re.sub(' ' + suffixes + '[.]', ' \\1<prd>', text)
    text = re.sub(' ' + alphabets + '[.]', ' \\1<prd>', text)
    text = re.sub(digits + '[.]' + digits, '\\1<prd>\\2', text)
    if '”' in text:
        text = text.replace('.”', '”.')
    if '"' in text:
        text = text.replace('."', '".')
    if '!' in text:
        text = text.replace('!"', '"!')
    if '?' in text:
        text = text.replace('?"', '"?')
    text = text.replace('.', '.<stop>')
    text = text.replace('?', '?<stop>')
    text = text.replace('!', '!<stop>')
    text = text.replace('<prd>', '.')
    sentences = text.split('<stop>')
    sentences = sentences[:-1]
    sentences = [cleaning(s.strip()) for s in sentences if len(s) > 10]
    return sentences

In [6]:
import json

with open(news[0]) as fopen:
    x = json.load(fopen)
    
x[0]

{'title': 'Lawatan Putera Mahkota Arab Saudi bincang pelaburan, ekonomi',
 'url': 'https://www.bharian.com.my/berita/nasional/2019/02/530861/lawatan-putera-mahkota-arab-saudi-bincang-pelaburan-ekonomi',
 'authors': ['Oleh Fahmy A Rosli'],
 'top-image': 'https://assets.bharian.com.my/images/articles/bhlawat14-O_BHfield_image_listing_featured.var_1550138917.jpg',
 'text': 'KUALA LUMPUR: Lawatan Putera Mahkota Arab Saudi, Mohammed bin Salman ke Malaysia pada Ahad ini terarah kepada pertemuan bagi membincangkan isu ekonomi dan pelaburan antara kedua-dua negara.\n\nMenteri Luar, Datuk Saifuddin Abdullah, berkata tujuan lawatan kerabat diraja Arab Saudi itu akan diperjelaskan oleh Kementerian Hal Ehwal Ekonomi.\n\n“Saya rasa elok perkara itu (lawatan Putera Mahkota Arab Saudi) dijelaskan sendiri oleh (Menteri Hal Ehwal Ekonomi) Datuk Seri Mohamed Azmin (Ali).\n\n“Selain saya (keluarkan kenyataan mengenai lawatan itu), kamu (pemberita) juga boleh ajukan soalan kepada Datuk Seri Mohamed Azmin.

In [7]:
split_into_sentences(x[0]['text'])

['KUALA LUMPUR: Lawatan Putera Mahkota Arab Saudi, Mohammed bin Salman ke Malaysia pada Ahad ini terarah kepada pertemuan bagi membincangkan isu ekonomi dan pelaburan antara kedua-dua negara.',
 'Menteri Luar, Datuk Saifuddin Abdullah, berkata tujuan lawatan kerabat diraja Arab Saudi itu akan diperjelaskan oleh Kementerian Hal Ehwal Ekonomi.',
 '"Saya rasa elok perkara itu (lawatan Putera Mahkota Arab Saudi) dijelaskan sendiri oleh (Menteri Hal Ehwal Ekonomi) Datuk Seri Mohamed Azmin (Ali).',
 '"Selain saya (keluarkan kenyataan mengenai lawatan itu), kamu (pemberita) juga boleh ajukan soalan kepada Datuk Seri Mohamed Azmin.',
 '"Ia banyak berkisar ekonomi dan perniagaan," katanya kepada pemberita selepas hadir di majlis meja bulat bersama belia di Institut Diplomasi dan Hubungan Luar Negeri, di sini, hari ini.',
 'Lawatan Putera Mahkota Arab Saudi ke Malaysia akan berlangsung selama dua hari berakhir Isnin ini.']

In [8]:
from tqdm import tqdm

texts = []

for n in tqdm(range(len(news))):
    n = news[n]
    with open(n) as fopen:
        s = json.load(fopen)
    
    for i in s:
        texts.extend([i['title']] + split_into_sentences(i['text']))
        texts.append('')

100%|██████████| 461/461 [01:36<00:00,  4.77it/s]


In [9]:
len(texts)

1479303

In [10]:
texts[:10]

['Lawatan Putera Mahkota Arab Saudi bincang pelaburan, ekonomi',
 'KUALA LUMPUR: Lawatan Putera Mahkota Arab Saudi, Mohammed bin Salman ke Malaysia pada Ahad ini terarah kepada pertemuan bagi membincangkan isu ekonomi dan pelaburan antara kedua-dua negara.',
 'Menteri Luar, Datuk Saifuddin Abdullah, berkata tujuan lawatan kerabat diraja Arab Saudi itu akan diperjelaskan oleh Kementerian Hal Ehwal Ekonomi.',
 '"Saya rasa elok perkara itu (lawatan Putera Mahkota Arab Saudi) dijelaskan sendiri oleh (Menteri Hal Ehwal Ekonomi) Datuk Seri Mohamed Azmin (Ali).',
 '"Selain saya (keluarkan kenyataan mengenai lawatan itu), kamu (pemberita) juga boleh ajukan soalan kepada Datuk Seri Mohamed Azmin.',
 '"Ia banyak berkisar ekonomi dan perniagaan," katanya kepada pemberita selepas hadir di majlis meja bulat bersama belia di Institut Diplomasi dan Hubungan Luar Negeri, di sini, hari ini.',
 'Lawatan Putera Mahkota Arab Saudi ke Malaysia akan berlangsung selama dua hari berakhir Isnin ini.',
 '',
 'I

In [11]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/news-30k/news-30k.json.zip

In [12]:
# !unzip news-30k.json.zip

In [13]:
with open('news-30k.json') as fopen:
    news = json.load(fopen)
    
news[0]

{'title': 'Ibu saudara Haziq terharu sokongan rakyat Malaysia',
 'url': 'https://www.themalaysianinsight.com/bahasa/s/142491',
 'news': 'themalaysianinsight',
 'language': 'malay',
 'top-image': 'https://www.themalaysianinsight.com/resources/stories_images/142491/perhimpunanan_solidarity_kedamaian_03__full.jpg',
 'text': 'WARIS keluarga Allahyarham Muhammad Haziq Mohd Tarmizi, 17, yang terkorban dalam tragedi tembakan di Christchurch, New Zealand, pada 15 Mac lepas, melahirkan rasa terharu akan sokongan diberi rakyat Malaysia semasa perhimpunan Solidariti Kedamaian.\n\nZarina Shuib , ibu saudara Muhammad Haziq, memanjatkan kesyukuran kepada Allah SWT kerana berkesempatan menyertai rakyat Malaysia dalam perhimpunan itu di Kuala Lumpur hari ini.',
 'date': '2019-03-23T03:52:02',
 'date_utc': '2019-03-22T19:52:02'}

In [14]:
for n in tqdm(range(len(news))):
    n = news[n]
    texts.extend([n['title']] + split_into_sentences(n['text']))
    texts.append('')

100%|██████████| 29855/29855 [00:26<00:00, 1138.11it/s]


In [15]:
texts[-20:]

['',
 'Transaksi mata wang digital bernilai RM75 juta sebulan',
 ': Malaysia menyaksikan transaksi sebanyak RM75 juta sebulan diproses melalui platform mata wang digital, kata Bank Negara Malaysia (BNM).',
 'Timbalan Gabenor, Abdul Rasheed Ghaffour berkata, nilai itu masih kecil berbanding dengan permodalan pasaran mata wang digital di seluruh dunia yang berjumlah AS$420 bilion (AS$1=RM4.07).',
 'Bagaimanapun, beliau berkata, BNM mengambil langkah ke hadapan dengan merancang untuk memperkenalkan dasar obligasi pelaporan yang akan menjadikan mata wang digital lebih telus dan untuk mengelak penyalahgunaan mata wang digital untuk pengubahan wang haram dan pembiayaan keganasan awal tahun depan".',
 'Dengan adanya obligasi pelaporan itu akan menjadikan bursa mata wang digital sebagai institusi pelaporan di bawah Akta Pencegahan Pengubahan Wang Haram, Pencegahan Pembiayaan Keganasan dan Hasil Daripada Aktiviti Haram 2001 (AMLA)," katanya kepada pemberita pada majlis taklimat di sini, hari in

In [16]:
with open('news-text.txt', 'w') as fopen:
    fopen.write('\n'.join(texts))